In [16]:
import sys
import pandas as pd
from pathlib import Path
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
sys.path.append("..")
from sif_src.loader import download_ms_marco, load_data, save_processed_data
from sif_src.preprocess_data import preprocess_text
from sif_src.utils import load_glove, load_word_vectors
from sif_src.sif import compute_word_frequencies, compute_sif_weights, compute_sif_embeddings, remove_pc

- Extract and serialize MS MARCO dataset

In [2]:
sys.path.append("..")

dataset = load_dataset('ms_marco', ignore_verifications=True)

train_dataset = dataset['train']
validation_dataset = dataset['validation']

/Users/yegor/conda/anaconda3/lib/python3.11/site-packages/datasets/load.py:1748: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(
Found cached dataset parquet (/Users/yegor/.cache/huggingface/datasets/parquet/v1.1-3b5a967d10b189e3/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
train_df = pd.DataFrame(train_dataset)
validation_df = pd.DataFrame(validation_dataset)

In [20]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/yegor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yegor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
def preprocess_pandas(text_series):

    text_series = text_series.str.lower()
    text_series = text_series.apply(word_tokenize)
    stop_words = set(stopwords.words('english'))

    text_series = text_series.apply(
        lambda tokens: [word for word in tokens if word.isalnum() and word not in stop_words]
    )
    return text_series


In [5]:
import pickle
from datetime import datetime

TIMESTAMP = datetime.now()
OUTPUT_PATH = Path("../pickle_backups")
TIMESTAMP_STR = TIMESTAMP.isoformat().replace(":", ".")

with open(OUTPUT_PATH / 
    f"marco_train_df{TIMESTAMP.strftime(r'%Y-%m-%dT%H.%M.%s')}.pickle",
    "wb"
) as f:
    pickle.dump(train_df, f)

'import pickle\nfrom datetime import datetime\n\nTIMESTAMP = datetime.now()\nOUTPUT_PATH = Path("../pickle_backups")\nTIMESTAMP_STR = TIMESTAMP.isoformat().replace(":", ".")\n\nwith open(OUTPUT_PATH / \n    f"marco_train_df{TIMESTAMP.strftime(r\'%Y-%m-%dT%H.%M.%s\')}.pickle",\n    "wb"\n) as f:\n    pickle.dump(train_df, f)'

In [6]:
with open(OUTPUT_PATH / 
    f"marco_valid_df{TIMESTAMP.strftime(r'%Y-%m-%dT%H.%M.%s')}.pickle",
    "wb"
) as f:
    pickle.dump(validation_df, f)

'with open(OUTPUT_PATH / \n    f"marco_valid_df{TIMESTAMP.strftime(r\'%Y-%m-%dT%H.%M.%s\')}.pickle",\n    "wb"\n) as f:\n    pickle.dump(validation_df, f)'